In [1]:
import pandas as pd
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
from efficientnet_pytorch import EfficientNet
import numpy as np
import timm
from PIL import Image
from PIL import ImageFile

In [2]:
import pytorch_lightning
pytorch_lightning.__version__

'1.1.0'

In [3]:
pytorch_lightning.utilities.seed.seed_everything(seed=42)

42

In [4]:
torch.__version__

'1.7.0'

In [5]:
import collections

In [6]:
def runSWA(modelDir=r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3__taylorLoss_timm_newhead'):

    for f in range(5):
        foldFolder = (os.path.join(modelDir,str(f)))

        print(f"----------- Fold: {f} -----------")
        weight1,weight2 = os.listdir(foldFolder)
        print(f"------------------------------------------------------------------")
        print(weight1,weight2)

        swa_state_dict = torch.load(os.path.join(foldFolder,weight1), map_location=lambda storage, loc: storage)['state_dict']
        newDict = collections.OrderedDict()
        
        for k,v in swa_state_dict.items():
            #print(k,v)
            if "model." in k:
#                 name = k.replace("model.","")
#                 swa_state_dict[k] = torch.zeros_like(v,dtype=torch.float32)
                newDict[k] = torch.zeros_like(v,dtype=torch.float32)
            
        for wts in [weight1,weight2]:
            checkpoint = os.path.join(foldFolder,wts)
            state_dict = torch.load(checkpoint, map_location=lambda storage, loc: storage)['state_dict']
            for k,v in state_dict.items():
                if "model." in k:
#                     name = k.replace("model.","")
#                     swa_state_dict[k] += v
                    newDict[k] += v
            #---
            #print(checkpoint)
        print('')

            #----
#         for k,v in swa_state_dict.items():
#             swa_state_dict[k] /= 2
            
        for k,v in newDict.items():
            newDict[k] /= 2
            

        avgWeightPath = os.path.join(modelDir,"avgWeights")

        if not os.path.exists(avgWeightPath):
            os.makedirs(avgWeightPath)
        name = avgWeightPath+f"\_Avg_fold_upd_{f}.ckpt"
#         torch.save(swa_state_dict, name)
        torch.save(newDict, name)


    del swa_state_dict,state_dict,;gc.collect()

In [7]:
runSWA(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation')

----------- Fold: 0 -----------
------------------------------------------------------------------
epoch=14_valid_loss=0.4199_valid_accuracy=0.8944.ckpt epoch=9_valid_loss=0.4155_valid_accuracy=0.8946.ckpt

----------- Fold: 1 -----------
------------------------------------------------------------------
epoch=10_valid_loss=0.3979_valid_accuracy=0.9021.ckpt epoch=6_valid_loss=0.4070_valid_accuracy=0.9007.ckpt

----------- Fold: 2 -----------
------------------------------------------------------------------
epoch=10_valid_loss=0.3749_valid_accuracy=0.8988.ckpt epoch=9_valid_loss=0.3755_valid_accuracy=0.8988.ckpt

----------- Fold: 3 -----------
------------------------------------------------------------------
epoch=10_valid_loss=0.3827_valid_accuracy=0.8827.ckpt epoch=7_valid_loss=0.3769_valid_accuracy=0.8832.ckpt

----------- Fold: 4 -----------
------------------------------------------------------------------
epoch=11_valid_loss=0.3743_valid_accuracy=0.8876.ckpt epoch=9_valid_loss=